#**스마트폰 센서 데이터 기반 모션 분류**
# 단계3 : 단계별 모델링


## 0.미션

단계별로 나눠서 모델링을 수행하고자 합니다.  

* 단계1 : 정적(0), 동적(1) 행동 분류 모델 생성
* 단계2 : 세부 동작에 대한 분류모델 생성
    * 단계1 모델에서 0으로 예측 -> 정적 행동 3가지 분류 모델링
    * 단계1 모델에서 1으로 예측 -> 동적 행동 3가지 분류 모델링
* 모델 통합
    * 두 단계 모델을 통합하고, 새로운 데이터에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
* 성능 비교
    * 기본 모델링의 성능과 비교
    * 모든 모델링은 [다양한 알고리즘 + 성능 튜닝]을 수행해야 합니다.


## 1.환경설정

### (1) 라이브러리 불러오기

* 세부 요구사항
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
    - 필요하다고 판단되는 라이브러리를 추가하세요.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 필요하다고 판단되는 라이브러리를 추가하세요.





### (2) 데이터 불러오기

* 주어진 데이터셋
    * data01_train.csv : 학습 및 검증용

 <br/>  

* 세부 요구사항
    - data01_train.csv 를 불러와 'data' 이름으로 저장합니다.
        - data에서 변수 subject는 삭제합니다.
    - data01_test.csv 를 불러와 'new_data' 이름으로 저장합니다.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# 전체 데이터 'data01_train.csv' 를 불러와 'data' 이름으로 저장
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KT AIVLE/5차 미프/실습파일/데이터/data01_train.csv')
new_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KT AIVLE/5차 미프/실습파일/데이터/data01_test.csv')

# 모델 변수 중요도 가져오기
import joblib
feature_importances = joblib.load('/content/drive/MyDrive/Colab Notebooks/KT AIVLE/5차 미프/실습파일/데이터/feature1.pkl')

In [11]:
# data에서 변수 subject는 삭제
data.drop(columns = 'subject', inplace=True)
new_data.drop(columns = 'subject', inplace=True)

#### 2) 기본 정보 조회

In [12]:
display(data.head())
display(new_data.head())

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity
0,0.288508,-0.009196,-0.103362,-0.988986,-0.962797,-0.967422,-0.989000,-0.962596,-0.965650,-0.929747,...,-0.487737,-0.816696,-0.042494,-0.044218,0.307873,0.072790,-0.601120,0.331298,0.165163,STANDING
1,0.265757,-0.016576,-0.098163,-0.989551,-0.994636,-0.987435,-0.990189,-0.993870,-0.987558,-0.937337,...,-0.237820,-0.693515,-0.062899,0.388459,-0.765014,0.771524,0.345205,-0.769186,-0.147944,LAYING
2,0.278709,-0.014511,-0.108717,-0.997720,-0.981088,-0.994008,-0.997934,-0.982187,-0.995017,-0.942584,...,-0.535287,-0.829311,0.000265,-0.525022,-0.891875,0.021528,-0.833564,0.202434,-0.032755,STANDING
3,0.289795,-0.035536,-0.150354,-0.231727,-0.006412,-0.338117,-0.273557,0.014245,-0.347916,0.008288,...,-0.004012,-0.408956,-0.255125,0.612804,0.747381,-0.072944,-0.695819,0.287154,0.111388,WALKING
4,0.394807,0.034098,0.091229,0.088489,-0.106636,-0.388502,-0.010469,-0.109680,-0.346372,0.584131,...,-0.157832,-0.563437,-0.044344,-0.845268,-0.974650,-0.887846,-0.705029,0.264952,0.137758,WALKING_DOWNSTAIRS


,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity
0,0.284379,-0.021981,-0.116683,-0.992490,-0.979640,-0.963321,-0.992563,-0.977304,-0.958142,-0.938850,...,-0.509523,-0.850065,-0.018043,0.092304,0.074220,-0.714534,-0.671943,-0.018351,-0.185733,SITTING
1,0.277440,-0.028086,-0.118412,-0.996620,-0.927676,-0.972294,-0.997346,-0.931405,-0.971788,-0.939837,...,-0.210792,-0.613367,-0.022456,-0.155414,0.247498,-0.112257,-0.826816,0.184489,-0.068699,STANDING
2,0.305833,-0.041023,-0.087303,0.006880,0.182800,-0.237984,0.005642,0.028616,-0.236474,0.016311,...,0.579587,0.394388,-0.362616,0.171069,0.576349,-0.688314,-0.743234,0.272186,0.053101,WALKING
3,0.276053,-0.016487,-0.108381,-0.995379,-0.983978,-0.975854,-0.995877,-0.985280,-0.974907,-0.941425,...,-0.566291,-0.841455,0.289548,0.079801,-0.020033,0.291898,-0.639435,-0.111998,-0.123298,SITTING
4,0.271998,0.016904,-0.078856,-0.973468,-0.702462,-0.869450,-0.979810,-0.711601,-0.856807,-0.920760,...,0.447577,0.214219,0.010111,0.114179,-0.830776,-0.325098,-0.840817,0.116237,-0.096615,STANDING


In [5]:
print(data.shape)
print(new_data.shape)

(5881, 562)
(1471, 562)


In [6]:
print(data.info())
print(new_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5881 entries, 0 to 5880
Columns: 562 entries, tBodyAcc-mean()-X to Activity
dtypes: float64(561), object(1)
memory usage: 25.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1471 entries, 0 to 1470
Columns: 562 entries, tBodyAcc-mean()-X to Activity
dtypes: float64(561), object(1)
memory usage: 6.3+ MB
None


In [7]:
print(data.describe())
print(new_data.describe())

       tBodyAcc-mean()-X  tBodyAcc-mean()-Y  tBodyAcc-mean()-Z  \
count        5881.000000        5881.000000        5881.000000   
mean            0.274811          -0.017799          -0.109396   
std             0.067614           0.039422           0.058373   
min            -0.503823          -0.684893          -1.000000   
25%             0.262919          -0.024877          -0.121051   
50%             0.277154          -0.017221          -0.108781   
75%             0.288526          -0.010920          -0.098163   
max             1.000000           1.000000           1.000000   

       tBodyAcc-std()-X  tBodyAcc-std()-Y  tBodyAcc-std()-Z  tBodyAcc-mad()-X  \
count       5881.000000       5881.000000       5881.000000       5881.000000   
mean          -0.603138         -0.509815         -0.604058         -0.628151   
std            0.448807          0.501815          0.417319          0.424345   
min           -1.000000         -0.999844         -0.999667         -1.000000   


## 2.데이터 전처리

* 세부 요구사항
    - Label 추가 : data 에 Activity_dynamic 를 추가합니다. Activity_dynamic은 과제1에서 is_dynamic과 동일한 값입니다.
    - x와 y1, y2로 분할하시오.
        * y1 : Activity
        * y2 : Activity_dynamic
    - train : val = 8 : 2 혹은 7 : 3
    - random_state 옵션을 사용하여 다른 모델과 비교를 위해 성능이 재현되도록 합니다.

In [13]:
non_dynamic = ['SITTING', 'STANDING', 'LAYING']
data['Activity_dynamic'] = data['Activity'].map(lambda x: 0 if x in non_dynamic else 1)
data

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity,Activity_dynamic
0,0.288508,-0.009196,-0.103362,-0.988986,-0.962797,-0.967422,-0.989000,-0.962596,-0.965650,-0.929747,...,-0.816696,-0.042494,-0.044218,0.307873,0.072790,-0.601120,0.331298,0.165163,STANDING,0
1,0.265757,-0.016576,-0.098163,-0.989551,-0.994636,-0.987435,-0.990189,-0.993870,-0.987558,-0.937337,...,-0.693515,-0.062899,0.388459,-0.765014,0.771524,0.345205,-0.769186,-0.147944,LAYING,0
2,0.278709,-0.014511,-0.108717,-0.997720,-0.981088,-0.994008,-0.997934,-0.982187,-0.995017,-0.942584,...,-0.829311,0.000265,-0.525022,-0.891875,0.021528,-0.833564,0.202434,-0.032755,STANDING,0
3,0.289795,-0.035536,-0.150354,-0.231727,-0.006412,-0.338117,-0.273557,0.014245,-0.347916,0.008288,...,-0.408956,-0.255125,0.612804,0.747381,-0.072944,-0.695819,0.287154,0.111388,WALKING,1
4,0.394807,0.034098,0.091229,0.088489,-0.106636,-0.388502,-0.010469,-0.109680,-0.346372,0.584131,...,-0.563437,-0.044344,-0.845268,-0.974650,-0.887846,-0.705029,0.264952,0.137758,WALKING_DOWNSTAIRS,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5876,0.277194,-0.012389,-0.131974,-0.994046,-0.940578,-0.917337,-0.994261,-0.932830,-0.908088,-0.936219,...,-0.690363,-0.034888,-0.261437,-0.391477,-0.877612,-0.912365,0.114009,0.080146,SITTING,0
5877,0.191568,0.013328,-0.105174,-0.126969,-0.121729,-0.327480,-0.192523,-0.109923,-0.295286,0.078644,...,-0.879215,0.721718,0.623151,0.866858,-0.445660,-0.690278,0.303194,-0.044188,WALKING_UPSTAIRS,1
5878,0.267981,-0.018348,-0.107440,-0.991303,-0.989881,-0.990313,-0.992386,-0.988852,-0.991237,-0.936099,...,-0.886851,0.060173,0.228739,0.684400,-0.216665,0.620363,-0.437247,-0.571840,LAYING,0
5879,0.212787,-0.048130,-0.121001,-0.041373,0.052449,-0.585361,-0.100714,0.023353,-0.554707,0.219814,...,-0.053556,0.260880,0.551742,-0.943773,-0.862899,-0.718009,0.292856,0.024920,WALKING_UPSTAIRS,1


In [14]:
# x, y로 나누기
y1 = data['Activity']
y2 = data['Activity_dynamic']
x = data.drop(columns = ['Activity_dynamic', 'Activity'])

In [15]:
# train과 validation 분할 수행
from sklearn.model_selection import train_test_split
X_train_dynamic, X_val_dynamic, y_train_dynamic, y_val_dynamic = train_test_split(x, y2, stratify = y2, random_state=42, test_size=0.25)

## **3.단계별 모델링**

![](https://github.com/DA4BAM/image/blob/main/step%20by%20step.png?raw=true)

### (1) 단계1 : 정적/동적 행동 분류 모델

* 세부 요구사항
    * 정적 행동(Laying, Sitting, Standing)과 동적 행동(동적 : Walking, Walking-Up, Walking-Down)을 구분하는 모델 생성.
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

#### 1) 알고리즘1 :

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

clf_random_forest = RandomForestClassifier(random_state=42)
clf_random_forest.fit(X_train_dynamic, y_train_dynamic)

predicted_random_forest = clf_random_forest.predict(X_val_dynamic)
macro_f1_score_random_forest = f1_score(y_val_dynamic, predicted_random_forest, average='macro')
print("RandomForestClassifier Accuracy Score:", np.mean(predicted_random_forest == y_val_dynamic))
print("RandomForestClassifier Macro F1 Score:", macro_f1_score_random_forest)

RandomForestClassifier Accuracy Score: 0.9993201903467029
RandomForestClassifier Macro F1 Score: 0.9993134269145051


#### 2) 알고리즘2 :

In [20]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score

clf_SGD = SGDClassifier(random_state=42)
clf_SGD.fit(X_train_dynamic, y_train_dynamic)

predicted = clf_SGD.predict(X_val_dynamic)
macro_f1_score = f1_score(y_val_dynamic, predicted, average='macro')
print('Accuracy Score :', np.mean(predicted == y_val_dynamic))
print("Macro F1 Score:", macro_f1_score)

Accuracy Score : 0.9993201903467029
Macro F1 Score: 0.9993134269145051


In [21]:
from sklearn.linear_model import LogisticRegression

clf_logistic = LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000)
clf_logistic.fit(X_train_dynamic, y_train_dynamic)

predicted_logistic = clf_logistic.predict(X_val_dynamic)
macro_f1_score_logistic = f1_score(y_val_dynamic, predicted_logistic, average='macro')
print("Accuracy Score:", np.mean(predicted_logistic == y_val_dynamic))
print("Macro F1 Score:", macro_f1_score_logistic)

Accuracy Score: 0.9993201903467029
Macro F1 Score: 0.9993134269145051


In [22]:
from sklearn.svm import SVC

clf_svm = SVC(kernel='linear', random_state=42)
clf_svm.fit(X_train_dynamic, y_train_dynamic)

predicted_svm = clf_svm.predict(X_val_dynamic)
macro_f1_score_svm = f1_score(y_val_dynamic, predicted_svm, average='macro')
print("SVM Accuracy Score:", np.mean(predicted_svm == y_val_dynamic))
print("SVM Macro F1 Score:", macro_f1_score_svm)


SVM Accuracy Score: 1.0
SVM Macro F1 Score: 1.0


### (2) 단계2-1 : 정적 동작 세부 분류

* 세부 요구사항
    * 정적 행동(Laying, Sitting, Standing)인 데이터 추출
    * Laying, Sitting, Standing 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

In [23]:
data_Activity_dynamic = data[data['Activity_dynamic'] == 0]
data_Activity_dynamic

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity,Activity_dynamic
0,0.288508,-0.009196,-0.103362,-0.988986,-0.962797,-0.967422,-0.989000,-0.962596,-0.965650,-0.929747,...,-0.816696,-0.042494,-0.044218,0.307873,0.072790,-0.601120,0.331298,0.165163,STANDING,0
1,0.265757,-0.016576,-0.098163,-0.989551,-0.994636,-0.987435,-0.990189,-0.993870,-0.987558,-0.937337,...,-0.693515,-0.062899,0.388459,-0.765014,0.771524,0.345205,-0.769186,-0.147944,LAYING,0
2,0.278709,-0.014511,-0.108717,-0.997720,-0.981088,-0.994008,-0.997934,-0.982187,-0.995017,-0.942584,...,-0.829311,0.000265,-0.525022,-0.891875,0.021528,-0.833564,0.202434,-0.032755,STANDING,0
7,0.272026,-0.001329,-0.125491,-0.992068,-0.912985,-0.972451,-0.994752,-0.943141,-0.976428,-0.925446,...,-0.704995,-0.024442,0.076332,0.741277,0.729812,-0.817201,0.037746,0.136129,STANDING,0
8,0.284338,0.021956,-0.006925,-0.980153,-0.838394,-0.782357,-0.983683,-0.816199,-0.743923,-0.914011,...,-0.400197,0.021212,-0.009465,-0.282762,0.563343,-0.782072,0.242834,-0.025285,STANDING,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5874,0.257476,-0.413865,0.017374,-0.919666,0.075259,-0.630716,-0.937986,0.176796,-0.613510,-0.866296,...,-0.091605,-0.000793,0.267189,-0.186202,0.098099,0.792970,-0.034020,-0.928148,LAYING,0
5875,0.277378,-0.013298,-0.104322,-0.996596,-0.987491,-0.973345,-0.996372,-0.987746,-0.973512,-0.942156,...,-0.956239,0.122320,0.136275,-0.708377,-0.507788,-0.818263,0.222620,0.035430,STANDING,0
5876,0.277194,-0.012389,-0.131974,-0.994046,-0.940578,-0.917337,-0.994261,-0.932830,-0.908088,-0.936219,...,-0.690363,-0.034888,-0.261437,-0.391477,-0.877612,-0.912365,0.114009,0.080146,SITTING,0
5878,0.267981,-0.018348,-0.107440,-0.991303,-0.989881,-0.990313,-0.992386,-0.988852,-0.991237,-0.936099,...,-0.886851,0.060173,0.228739,0.684400,-0.216665,0.620363,-0.437247,-0.571840,LAYING,0


In [24]:
# x, y로 나누기
y2 = data_Activity_dynamic['Activity']
x = data_Activity_dynamic.drop(columns = ['Activity_dynamic', 'Activity'])
# train과 validation 분할 수행
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(x, y2, stratify = y2, random_state=42, test_size=0.25)

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

clf_random_forest = RandomForestClassifier(random_state=42)
clf_random_forest.fit(X_train, y_train)

predicted_random_forest = clf_random_forest.predict(X_val)
macro_f1_score_random_forest = f1_score(y_val, predicted_random_forest, average='macro')
print("RandomForestClassifier Accuracy Score:", np.mean(predicted_random_forest == y_val))
print("RandomForestClassifier Macro F1 Score:", macro_f1_score_random_forest)

RandomForestClassifier Accuracy Score: 0.9728059332509271
RandomForestClassifier Macro F1 Score: 0.9723128505416329


In [26]:
from sklearn.linear_model import LogisticRegression

clf_logistic = LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000)
clf_logistic.fit(X_train, y_train)

predicted_logistic = clf_logistic.predict(X_val)
macro_f1_score_logistic = f1_score(y_val, predicted_logistic, average='macro')
print("Accuracy Score:", np.mean(predicted_logistic == y_val))
print("Macro F1 Score:", macro_f1_score_logistic)

Accuracy Score: 0.9777503090234858
Macro F1 Score: 0.9773399398865158


In [27]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score

clf_SGD = SGDClassifier(random_state=42)
clf_SGD.fit(X_train, y_train)

predicted = clf_SGD.predict(X_val)
macro_f1_score = f1_score(y_val, predicted, average='macro')
print('Accuracy Score :', np.mean(predicted == y_val))
print("Macro F1 Score:", macro_f1_score)

Accuracy Score : 0.9616810877626699
Macro F1 Score: 0.9609937723133148


In [28]:
from sklearn.svm import SVC

clf_svm = SVC(kernel='linear', random_state=42)
clf_svm.fit(X_train, y_train)

predicted_svm = clf_svm.predict(X_val)
macro_f1_score_svm = f1_score(y_val, predicted_svm, average='macro')
print("SVM Accuracy Score:", np.mean(predicted_svm == y_val))
print("SVM Macro F1 Score:", macro_f1_score_svm)


SVM Accuracy Score: 0.9802224969097652
SVM Macro F1 Score: 0.9798601611187111


### (3) 단계2-2 : 동적 동작 세부 분류

* 세부 요구사항
    * 동적 행동(Walking, Walking Upstairs, Walking Downstairs)인 데이터 추출
    * Walking, Walking Upstairs, Walking Downstairs 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

In [29]:
data_Activity_dynamic = data[data['Activity_dynamic'] == 1]
data_Activity_dynamic

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity,Activity_dynamic
3,0.289795,-0.035536,-0.150354,-0.231727,-0.006412,-0.338117,-0.273557,0.014245,-0.347916,0.008288,...,-0.408956,-0.255125,0.612804,0.747381,-0.072944,-0.695819,0.287154,0.111388,WALKING,1
4,0.394807,0.034098,0.091229,0.088489,-0.106636,-0.388502,-0.010469,-0.109680,-0.346372,0.584131,...,-0.563437,-0.044344,-0.845268,-0.974650,-0.887846,-0.705029,0.264952,0.137758,WALKING_DOWNSTAIRS,1
5,0.330708,0.007561,-0.061371,-0.215760,0.101075,0.072949,-0.269857,0.060060,0.101298,-0.019263,...,-0.887024,-0.030645,-0.852091,-0.500195,0.306091,-0.552729,0.253885,0.291256,WALKING_UPSTAIRS,1
6,0.121465,-0.031902,-0.005196,-0.152198,-0.113104,-0.239423,-0.202401,-0.164698,-0.247099,0.114668,...,-0.775779,0.445206,-0.003487,-0.940185,0.041387,-0.886603,0.173338,-0.005627,WALKING,1
12,0.303885,0.002768,-0.038613,-0.168656,0.190336,-0.140473,-0.205134,0.101144,-0.120572,-0.000818,...,-0.329728,-0.040030,0.257252,0.076091,-0.123425,-0.752882,0.266729,0.045692,WALKING,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5868,0.277709,-0.053919,-0.098746,-0.589970,-0.076626,-0.500837,-0.605474,-0.142798,-0.506696,-0.533485,...,-0.268237,-0.076922,0.706620,-0.954969,-0.324779,-0.691663,0.153974,-0.196833,WALKING,1
5869,0.252496,-0.009773,-0.118293,-0.155168,-0.244513,-0.044524,-0.244367,-0.229509,-0.042199,0.291682,...,-0.752980,0.619246,-0.684483,0.924623,-0.353041,-0.775518,0.253218,0.013216,WALKING_UPSTAIRS,1
5873,0.264961,-0.034719,-0.088140,-0.368158,-0.074514,-0.229606,-0.394387,-0.082437,-0.243525,-0.261087,...,-0.046157,0.037517,-0.903692,0.614933,-0.610918,-0.601604,0.335996,0.156518,WALKING,1
5877,0.191568,0.013328,-0.105174,-0.126969,-0.121729,-0.327480,-0.192523,-0.109923,-0.295286,0.078644,...,-0.879215,0.721718,0.623151,0.866858,-0.445660,-0.690278,0.303194,-0.044188,WALKING_UPSTAIRS,1


In [30]:
# x, y로 나누기
y2 = data_Activity_dynamic['Activity']
x = data_Activity_dynamic.drop(columns = ['Activity_dynamic', 'Activity'])

# train과 validation 분할 수행
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(x, y2, stratify = y2, random_state=42, test_size=0.25)

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

clf_random_forest = RandomForestClassifier(random_state=42)
clf_random_forest.fit(X_train, y_train)

predicted_random_forest = clf_random_forest.predict(X_val)
macro_f1_score_random_forest = f1_score(y_val, predicted_random_forest, average='macro')
print("RandomForestClassifier Accuracy Score:", np.mean(predicted_random_forest == y_val))
print("RandomForestClassifier Macro F1 Score:", macro_f1_score_random_forest)

RandomForestClassifier Accuracy Score: 0.9803625377643505
RandomForestClassifier Macro F1 Score: 0.9798993784956286


In [32]:
from sklearn.linear_model import LogisticRegression

clf_logistic = LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000)
clf_logistic.fit(X_train, y_train)

predicted_logistic = clf_logistic.predict(X_val)
macro_f1_score_logistic = f1_score(y_val, predicted_logistic, average='macro')
print("Accuracy Score:", np.mean(predicted_logistic == y_val))
print("Macro F1 Score:", macro_f1_score_logistic)

Accuracy Score: 0.9909365558912386
Macro F1 Score: 0.9909067674678989


In [33]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score

clf_SGD = SGDClassifier(random_state=42)
clf_SGD.fit(X_train, y_train)

predicted = clf_SGD.predict(X_val)
macro_f1_score = f1_score(y_val, predicted, average='macro')
print('Accuracy Score :', np.mean(predicted == y_val))
print("Macro F1 Score:", macro_f1_score)

Accuracy Score : 0.9879154078549849
Macro F1 Score: 0.9878934828958946


In [34]:
from sklearn.svm import SVC

clf_svm = SVC(kernel='linear', random_state=42)
clf_svm.fit(X_train, y_train)

predicted_svm = clf_svm.predict(X_val)
macro_f1_score_svm = f1_score(y_val, predicted_svm, average='macro')
print("SVM Accuracy Score:", np.mean(predicted_svm == y_val))
print("SVM Macro F1 Score:", macro_f1_score_svm)

SVM Accuracy Score: 0.9954682779456193
SVM Macro F1 Score: 0.9954222989755795


### (4) 분류 모델 합치기


* 세부 요구사항
    * 두 단계 모델을 통합하고, 새로운 데이터(test)에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
    * 데이터 파이프라인 구축 : test데이터가 로딩되어 전처리 과정을 거치고, 예측 및 성능 평가 수행

![](https://github.com/DA4BAM/image/blob/main/pipeline%20function.png?raw=true)

#### 1) 함수 만들기

In [ ]:
# 학습 및 평가 데이터 불러옴
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KT AIVLE/5차 미프/실습파일/데이터/data01_train.csv')
new_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KT AIVLE/5차 미프/실습파일/데이터/data01_test.csv')

# data에서 변수 subject는 삭제
data.drop(columns = 'subject', inplace=True)
new_data.drop(columns = 'subject', inplace=True)

non_dynamic = ['SITTING', 'STANDING', 'LAYING']
data['Activity_dynamic'] = data['Activity'].map(lambda x: 0 if x in non_dynamic else 1)

# 1번 모델을 위해 x, y로 나누기
y1 = data['Activity']
y2 = data['Activity_dynamic']
x = data.drop(columns = ['Activity_dynamic', 'Activity'])

# 1번 모델을 위해 train과 validation 분할 수행
X_train_dynamic, X_val_dynamic, y_train_dynamic, y_val_dynamic = train_test_split(x, y2, stratify = y2, random_state=42, test_size=0.25)

### 첫번째 모델 학습
clf_svm = SVC(kernel='linear', random_state=42)
clf_svm.fit(X_train_dynamic, y_train_dynamic)

######################################################

# 2-1을 위한 x, y나누기
data_Activity_dynamic = data[data['Activity_dynamic'] == 0]
y1 = data_Activity_dynamic['Activity']
x = data_Activity_dynamic.drop(columns = ['Activity_dynamic', 'Activity'])

X_train_Activity_0, X_val_Activity_0, y_train_Activity_0, y_val_Activity_0 = train_test_split(x, y1, stratify = y1, random_state=42, test_size=0.25)

### 두번째 모델 학습
clf_logistic = LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000)
clf_logistic.fit(X_train_Activity_0, y_train_Activity_0)

#######################################################

# 2-2을 위한 x, y나누기
data_Activity_dynamic = data[data['Activity_dynamic'] == 1]
y1 = data_Activity_dynamic['Activity']
x = data_Activity_dynamic.drop(columns = ['Activity_dynamic', 'Activity'])

X_train_Activity_1, X_val_Activity_1, y_train_Activity_1, y_val_Activity_1 = train_test_split(x, y1, stratify = y1, random_state=42, test_size=0.25)

##########################################################

### 세번째 모델 학습
clf_svm_Act_1 = SVC(kernel='linear', random_state=42)
clf_svm_Act_1.fit(X_train_Activity_1, y_train_Activity_1)


# 모델 평가
# predictions_svm = clf_svm.predict(X_val_dynamic)
# predictions_logistic = clf_logistic.predict(X_val_Activity_0)
# predictions_svm_Act_1 = clf_svm_Act_1.predict(X_val_dynamic)

from sklearn.ensemble import VotingClassifier

# 투표 모델 생성
voting_clf = VotingClassifier(estimators=[('svm', clf_svm), ('logistic', clf_logistic), ('svm_Act_1', clf_svm_Act_1)], voting='hard')

X_train, X_val, y_train, y_val = train_test_split(x, y1, stratify = y1, random_state=42, test_size=0.25)

# 투표 모델 학습
voting_clf.fit(X_val, y_val)

# 최종 예측
final_predictions = voting_clf.predict(X_val)

from sklearn.metrics import accuracy_score, f1_score

final_accuracy = accuracy_score(y_val, final_predictions)
final_macro_f1 = f1_score(y_val, final_predictions, average='macro')
print("Final Model Accuracy:", final_accuracy)
print("Final Model Macro F1 Score:", final_macro_f1)


# 4.최종 분류 모델 (전체 데이터 넣어서 모델 학습)

In [ ]:
# 학습 및 평가 데이터 불러옴
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KT AIVLE/5차 미프/실습파일/데이터/data01_train.csv')
new_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KT AIVLE/5차 미프/실습파일/데이터/data01_test.csv')

# data에서 변수 subject는 삭제
data.drop(columns = 'subject', inplace=True)
new_data.drop(columns = 'subject', inplace=True)

non_dynamic = ['SITTING', 'STANDING', 'LAYING']
data['Activity_dynamic'] = data['Activity'].map(lambda x: 0 if x in non_dynamic else 1)

# 1번 모델을 위해 x, y로 나누기
y1 = data['Activity']
y2 = data['Activity_dynamic']
x = data.drop(columns = ['Activity_dynamic', 'Activity'])

# 1번 모델을 위해 train과 validation 분할 수행
X_train_dynamic, X_val_dynamic, y_train_dynamic, y_val_dynamic = train_test_split(x, y2, stratify = y2, random_state=42, test_size=0.25)

### 첫번째 모델 학습
clf_svm = SVC(kernel='linear', random_state=42)
clf_svm.fit(X_train_dynamic, y_train_dynamic)

######################################################

# 2-1을 위한 x, y나누기
data_Activity_dynamic = data[data['Activity_dynamic'] == 0]
y1 = data_Activity_dynamic['Activity']
x = data_Activity_dynamic.drop(columns = ['Activity_dynamic', 'Activity'])

X_train_Activity_0, X_val_Activity_0, y_train_Activity_0, y_val_Activity_0 = train_test_split(x, y1, stratify = y1, random_state=42, test_size=0.25)

### 두번째 모델 학습
clf_logistic = LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000)
clf_logistic.fit(X_train_Activity_0, y_train_Activity_0)

#######################################################

# 2-2을 위한 x, y나누기
data_Activity_dynamic = data[data['Activity_dynamic'] == 1]
y1 = data_Activity_dynamic['Activity']
x = data_Activity_dynamic.drop(columns = ['Activity_dynamic', 'Activity'])

X_train_Activity_1, X_val_Activity_1, y_train_Activity_1, y_val_Activity_1 = train_test_split(x, y1, stratify = y1, random_state=42, test_size=0.25)

##########################################################

### 세번째 모델 학습
clf_svm_Act_1 = SVC(kernel='linear', random_state=42)
clf_svm_Act_1.fit(X_train_Activity_1, y_train_Activity_1)


# 모델 평가
# predictions_svm = clf_svm.predict(X_val_dynamic)
# predictions_logistic = clf_logistic.predict(X_val_Activity_0)
# predictions_svm_Act_1 = clf_svm_Act_1.predict(X_val_dynamic)

from sklearn.ensemble import VotingClassifier

# 투표 모델 생성
voting_clf = VotingClassifier(estimators=[('svm', clf_svm), ('logistic', clf_logistic), ('svm_Act_1', clf_svm_Act_1)], voting='hard')

X_train, X_val, y_train, y_val = train_test_split(x, y1, stratify = y1, random_state=42, test_size=0.25)

# 투표 모델 학습
voting_clf.fit(X_val, y_val)

# 최종 예측
final_predictions = voting_clf.predict(X_val)

from sklearn.metrics import accuracy_score, f1_score

final_accuracy = accuracy_score(y_val, final_predictions)
final_macro_f1 = f1_score(y_val, final_predictions, average='macro')
print("Final Model Accuracy:", final_accuracy)
print("Final Model Macro F1 Score:", final_macro_f1)
